In [ ]:
base_url = 'https://www.rxlist.com'

def scrape_alphabet_links():
    response = requests.get(f'{base_url}/drugs/alpha_a.htm')
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []

    for li in soup.select('div#A_Z ul li'):
        a = li.find('a')
        link = a['href']
        links.append(base_url + link)

    return links

def scrape_drug_links(alphabet_link):
    response = requests.get(alphabet_link)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []

    for li in soup.select('div.AZ_results ul li'):
        a = li.find('a')
        link = a['href']
        links.append(link)

    return links

def scrape_drug_details(drug_link):
    response = requests.get(drug_link)
    soup = BeautifulSoup(response.text, 'html.parser')

    details = {}

    for li in soup.select('div.toc.link ul.fda_toc li'):
        a = li.find('a')
        detail_name = a.text

        detail_link = drug_link + a['href'].split('#')[1]

        detail_response = requests.get(detail_link)
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        detail_text = detail_soup.text.strip()

        details[detail_name] = detail_text

    return details

def main():
    alphabet_links = scrape_alphabet_links()
    
    # Only use the first link, which should be the link to the 'A' page
    alphabet_link = alphabet_links[0]

    all_drugs = []

    for alphabet_link in alphabet_links:
        drug_links = scrape_drug_links(alphabet_link)

        for drug_link in drug_links:
            drug_details = scrape_drug_details(drug_link)
            all_drugs.append(drug_details)

    keys = all_drugs[0].keys()

    with open('drugs.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_drugs)

main()